# Word2Vec 词向量
将字词从文字空间映射到向量空间。  
Map words from text space to vector space.

每个字词都会有一个对应的代表其__语义__的向量。  
Each word has a corresponding vector which represents the __semantics__ of a word.

有多重方法做到这一点。  
Many methods can achieve such a goal.

有基于次数和平率的方法。  
There are count and frequency based methods.

本教程介绍的是论文《[Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)》中所使用的方法。  
This tutorial explains the method used in [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)

__为了最大地照顾到各种同学(不会中文的同学)，我主要用英文。关键句段会有中文。__  
__For maximum fellow learners reach（non-Chinese learners）, I will mostly use English. I will provide Chinese for some key phrases.__

__英文是在学界与开发者社区是一门重要的语言。中国的同学们多看英文资料也是有好处的。__  
__English is an important languge in academia and developer communities. Fellow learners in China can benefit a lot by reading English materials.__

### Note:
This tutorial is programming oriented. I will descripe it as 20% theory and 80% programming.

Therefore, I assume you already have understand the basics of __Deep Learning__.

I also assume that you have some experience with [__TensorFlow__](https://www.tensorflow.org/).

I strongly encourage you to read all materials linked at the end of this notebook or watch my videos before proceeding.

## A Gensim example
Besides the [TensorFlow example](word2vec.py), I show you a Gensim example here.

[Gensim](https://radimrehurek.com/gensim/) is a topic modelling toolkit. It has word2vec implemented.

In [3]:
### First, get the data
### We use the complete works of Shakespear



D:\Anaconda3\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## References
### Paper
[Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)  
[Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/pdf/1310.4546.pdf)  


### Other Tutorials
[A post by Chris McCormick](http://mccormickml.com/2016/04/27/word2vec-resources/)  
[Second post by Chris McCormick](http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/)  

### TensorFlow
[TensorFlow Tutorial](https://www.tensorflow.org/tutorials/word2vec)  
[TensorFlow Example Code](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/word2vec/word2vec_basic.py)  